In [1]:
import os
os.chdir('C:/Users/eddie/Desktop/Prof. Rumi Chunara/alcohol')
import sklearn
print(sklearn.__version__)
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from scipy.stats import uniform
from evaluation.metric import computeAccuracy, computeF1Score, computeAUC
from sklearn.metrics import roc_auc_score
from sklearn.grid_search import RandomizedSearchCV, GridSearchCV

from pipelines.alcohol import AlcoholPipeline
# iterate_heirarchy use class DataAccess
# from preprocessing import iterate_heirarchy
from classification.compute import CustomGridSearch
from scripts.gridsearch import text_grid

0.17


Prepare the alcohol data

In [30]:
df = pd.read_csv("./data/alcohol_training_instances.csv")
alc_labels = []
# extracting pre-labeled classification
for i in range(0,len(df.labels)):
    alc_labels.append(df.labels[i][12])
alc_labels = list(map(int, alc_labels))
print(alc_labels[:10])
print(len(alc_labels))
# print(type(alc_labels))
alc_labels = np.asarray(alc_labels)
print(type(alc_labels))
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, alc_labels, test_size=0.33, random_state=42)

[0, 1, 1, 1, 1, 1, 1, 1, 0, 1]
15651
<class 'numpy.ndarray'>


Create a model for the following grid search

In [31]:
clf_alc_LR = AlcoholPipeline(global_features=["text"]).pipeline(LogisticRegression())
# score() returns the accuracy
accuracy_clf_alc_LR = clf_alc_LR.fit(X_train, y_train).score(X_test, y_test)
print('accuracy: ', accuracy_clf_alc_LR)
print(type(accuracy_clf_alc_LR))

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


accuracy:  0.70648596321394
<class 'numpy.float64'>


Now we are trying to implement grid search.

In [32]:
# clf_alc_grid_LR = AlcoholPipeline(global_features=["text"]).pipeline(LogisticRegression())

param_grid = {
    'clf__C': uniform(0.0001, 1000),
#     'clf__penalty': ['l2', "l1"],
    'clf__penalty': ['l2'],
    'clf__tol': uniform(0.0001, 0.001),
    'clf__verbose': [0],
    'features__text__tfidf__analyzer': ['char'],
    'features__text__tfidf__ngram_range': [(2,5)]
}

param_grid.update(text_grid)

cv_kwargs = dict(
    n_iter=50,
    scoring=None,
    fit_params=None,
    n_jobs=-1,
    iid=True,
    refit=True,
    cv=10,
    verbose=0,
    pre_dispatch='2*n_jobs',
    error_score=0
)

In [33]:
clf_alc_LR_grid = RandomizedSearchCV(clf_alc_LR, param_grid, **cv_kwargs)
clf_alc_LR_grid.fit(X_train, y_train)
# clf_alc_LR_grid.fit(df, alc_labels)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


RandomizedSearchCV(cv=10, error_score=0,
          estimator=Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
          fit_params={}, iid=True, n_iter=50, n_jobs=-1,
          param_distributions={'clf__C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001D0A7CE7668>, 'clf__penalty': ['l2'], 'clf__tol': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001D0A7CE7EF0>, 'clf__verbose': [0], 'features__text__tfidf__analyzer': ['char', 'word'], 'featur..., 'english'], 'features__text__tfidf__tokenizer': [None, <function tokenize at 0x000001D0A5860268>]},
          pre_dispatch='2*n_jobs', random_state=None

In [34]:
# get the best estimator 
best_estimator_LR = clf_alc_LR_grid.best_estimator_
print('best estimator: ', best_estimator_LR)
print('-------------------------- mean accuracy on the cross validation of best estimator --------------------------')
print('best score: ', clf_alc_LR_grid.best_score_)
print('best score related parameters: ', clf_alc_LR_grid.best_params_)

best estimator:  Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=F...e,
          solver='liblinear', tol=0.0008666364659600043, verbose=0,
          warm_start=False))])
-------------------------- mean accuracy on the cross validation of best estimator --------------------------
best score:  0.8359717718863247
best score related parameters:  {'clf__C': 100.07857541229242, 'clf__penalty': 'l2', 'clf__tol': 0.0008666364659600043, 'clf__verbose': 0, 'features__text__tfidf__analyzer': 'char', 'features__text__tfidf__lowercase': False, 'features__text__tfidf__max_features': 80076, 'features__text__tfidf__min_df': 3, 'features__text__tfidf__ngram_range': (2, 5), 'features__text__tfidf__norm': 'l2', 'features__text__tfidf__stop_words': 

Test the best estimator performance on testing dataset

In [35]:
# predict in testing set.
y_alc_LR = best_estimator_LR.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_alc_LR, y_test)
f1_score_LR = computeF1Score(y_test, y_alc_LR)
print('F1 Score: ', f1_score_LR)
auc_score_LR = computeAUC(best_estimator_LR, X_test, y_test, plot=False)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('alcohol LR AUC: ', auc_score_LR)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


length of total comps:  5165
pretrained alcohol classifer:  835 different item in whole dataset
accuracy: 83.833495%
F1 Score:  0.8790030430372411
alcohol LR AUC:  0.8789322873226146


We can see the performance is still a little lower than the classifier with updated parameters

Plot the grid search result in table. From the 'best score' and 'best score realated parameters' above, we can see that best estimator has the highest mean_validation_score below.

In [36]:
cv = pd.DataFrame(clf_alc_LR_grid.grid_scores_)
cv.cv_validation_scores[0]
cv

,parameters,mean_validation_score,cv_validation_scores
0,"{'clf__C': 575.4621664586958, 'clf__penalty': ...",0.708373,"[0.7095238095238096, 0.7028571428571428, 0.718..."
1,"{'clf__C': 711.6501607479418, 'clf__penalty': ...",0.775892,"[0.7695238095238095, 0.8, 0.776930409914204, 0..."
2,"{'clf__C': 313.48523431533516, 'clf__penalty':...",0.794488,"[0.7904761904761904, 0.800952380952381, 0.7902..."
3,"{'clf__C': 718.8424870427268, 'clf__penalty': ...",0.780946,"[0.7828571428571428, 0.7828571428571428, 0.780..."
4,"{'clf__C': 299.52969284025437, 'clf__penalty':...",0.828629,"[0.8457142857142858, 0.8333333333333334, 0.821..."
5,"{'clf__C': 864.429914945513, 'clf__penalty': '...",0.683197,"[0.6828571428571428, 0.6819047619047619, 0.688..."
6,"{'clf__C': 424.08570915755354, 'clf__penalty':...",0.785237,"[0.7838095238095238, 0.7885714285714286, 0.786..."
7,"{'clf__C': 142.0570009305359, 'clf__penalty': ...",0.780565,"[0.7885714285714286, 0.7857142857142857, 0.780..."
8,"{'clf__C': 751.0622916503639, 'clf__penalty': ...",0.657257,"[0.6495238095238095, 0.66, 0.6825548141086749,..."
9,"{'clf__C': 100.07857541229242, 'clf__penalty':...",0.835972,"[0.8542857142857143, 0.8447619047619047, 0.829..."


Now we will trying to do grid search on the created classifier with updated parameters. 

Classifier with updated parameters. The reason why we choose these parameters please check alcoholClassiferLR.ipynb

In [37]:
clf_alc_LR_updateParams = AlcoholPipeline(global_features=["text"]).pipeline(LogisticRegression())
params_LR = {'features__text__tfidf__dtype':np.int64, 'features__text__tfidf__max_features':71233, 'features__text__tfidf__ngram_range':(2,5) 
            }
# to see all parameters in model
# clf_alc_LR_updateParams.get_params().keys()
clf_alc_LR_updateParams.set_params(**params_LR)

Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

Train the LR model with updated parameters, and predict in testing set

In [38]:
clf_alc_LR_updateParams.fit(X_train, y_train)
# predict in testing set.
y_alc_LR_updateParams = clf_alc_LR_updateParams.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_alc_LR_updateParams, y_test)
f1_score_LR_updateParams = computeF1Score(y_test, y_alc_LR_updateParams)
print('F1 Score: ', f1_score_LR_updateParams)
auc_score_LR_updateParams = computeAUC(clf_alc_LR_updateParams, X_test, y_test, plot=False)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('alcohol LR AUC: ', auc_score_LR_updateParams)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


length of total comps:  5165
pretrained alcohol classifer:  750 different item in whole dataset
accuracy: 85.479187%
F1 Score:  0.8954558126568163
alcohol LR AUC:  0.88876528818431


In [39]:
clf_alc_LR_updateParams.steps

[('features', FeatureUnion(n_jobs=1,
         transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=71233, min_df=1,
     ...      token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None))]))],
         transformer_weights=None)),
 ('scaler', Normalizer(copy=True, norm='l2')),
 ('clf',
  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
            penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
            verbose=0, warm_start=False))]

Grid search on classifier with updated parameters

In [40]:
clf_alc_LR_grid_updateParams = RandomizedSearchCV(clf_alc_LR_updateParams, param_grid, **cv_kwargs)
clf_alc_LR_grid_updateParams.fit(X_train, y_train)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


RandomizedSearchCV(cv=10, error_score=0,
          estimator=Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
          fit_params={}, iid=True, n_iter=50, n_jobs=-1,
          param_distributions={'clf__C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001D0A7CE7668>, 'clf__penalty': ['l2'], 'clf__tol': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001D0A7CE7EF0>, 'clf__verbose': [0], 'features__text__tfidf__analyzer': ['char', 'word'], 'featur..., 'english'], 'features__text__tfidf__tokenizer': [None, <function tokenize at 0x000001D0A5860268>]},
          pre_dispatch='2*n_jobs', random_state=None

In [41]:
# get the best estimator 
best_estimator_LR_updateParams = clf_alc_LR_grid_updateParams.best_estimator_
print('best estimator: ', best_estimator_LR_updateParams)
print('-------------------------- mean accuracy on the cross validation of best estimator --------------------------')
print('best score: ', clf_alc_LR_grid_updateParams.best_score_)
print('best score related parameters: ', clf_alc_LR_grid_updateParams.best_params_)
# best_estimator_LR_updateParams.steps[1]

best estimator:  Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...,
          solver='liblinear', tol=0.00018877962869388878, verbose=0,
          warm_start=False))])
-------------------------- mean accuracy on the cross validation of best estimator --------------------------
best score:  0.845317566278848
best score related parameters:  {'clf__C': 18.445333560750008, 'clf__penalty': 'l2', 'clf__tol': 0.00018877962869388878, 'clf__verbose': 0, 'features__text__tfidf__analyzer': 'char', 'features__text__tfidf__lowercase': True, 'features__text__tfidf__max_features': 95717, 'features__text__tfidf__min_df': 4, 'features__text__tfidf__ngram_range': (2, 5), 'features__text__tfidf__norm': 'l1', 'features__text__tfidf__stop_words': N

Test the best estimator performance on testing dataset (classifier with updated parameters)

In [42]:
# predict in testing set.
y_alc_LR_updateParams = best_estimator_LR_updateParams.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_alc_LR_updateParams, y_test)
f1_score_LR_updateParams = computeF1Score(y_test, y_alc_LR_updateParams)
print('F1 Score: ', f1_score_LR_updateParams)
auc_score_LR_updateParams = computeAUC(best_estimator_LR_updateParams, X_test, y_test, plot=False)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('alcohol LR AUC: ', auc_score_LR_updateParams)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


length of total comps:  5165
pretrained alcohol classifer:  806 different item in whole dataset
accuracy: 84.394966%
F1 Score:  0.8839953943580886
alcohol LR AUC:  0.8845617675054297


Plot the grid search result in table (classifier with updated parameters)

In [43]:
cv_updateParams = pd.DataFrame(clf_alc_LR_grid_updateParams.grid_scores_)
cv_updateParams.cv_validation_scores[0]
cv_updateParams

,parameters,mean_validation_score,cv_validation_scores
0,"{'clf__C': 72.09216253227684, 'clf__penalty': ...",0.707324,"[0.7057142857142857, 0.7038095238095238, 0.717..."
1,"{'clf__C': 809.6981930135545, 'clf__penalty': ...",0.829296,"[0.8438095238095238, 0.8419047619047619, 0.824..."
2,"{'clf__C': 795.0562856131112, 'clf__penalty': ...",0.668320,"[0.6619047619047619, 0.6571428571428571, 0.684..."
3,"{'clf__C': 478.0389626305434, 'clf__penalty': ...",0.667271,"[0.679047619047619, 0.6638095238095238, 0.6634..."
4,"{'clf__C': 550.4385896880151, 'clf__penalty': ...",0.790387,"[0.7895238095238095, 0.7961904761904762, 0.788..."
5,"{'clf__C': 226.6252854747271, 'clf__penalty': ...",0.830250,"[0.84, 0.8380952380952381, 0.8427073403241182,..."
6,"{'clf__C': 959.1307458484556, 'clf__penalty': ...",0.824814,"[0.8342857142857143, 0.8438095238095238, 0.819..."
7,"{'clf__C': 496.9457401966268, 'clf__penalty': ...",0.824909,"[0.84, 0.8304761904761905, 0.8303145853193518,..."
8,"{'clf__C': 781.6531254516618, 'clf__penalty': ...",0.786573,"[0.7790476190476191, 0.7942857142857143, 0.779..."
9,"{'clf__C': 99.4611969149653, 'clf__penalty': '...",0.834351,"[0.8504761904761905, 0.8428571428571429, 0.840..."


It seems the classifier with updated parameters is already the best we can get from grid search. After update specific parameters, model is hard to find the best parameters in grid search, especially when the interation is less.